In [4]:
%load_ext autoreload
%autoreload 2
import sys; sys.path.append("../src")  # adjust if needed
sys.path.append("..")
sys.path.append("../src/simkit")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from simkit import (
    TradeLedger, preview_expiries, expire_trades_vectorized,
    daily_pnl_curve, realized_pnl_total,
    generate_straddle_trades
)

Load checkpoint + reconstruct index + pick test split

In [13]:
import os, json, yaml, joblib, numpy as np, pandas as pd
from sklearn.metrics import roc_auc_score
from src.checkpoint_storage import load_all_checkpoints_v2


from src.load_data import  prepare_volatility_data
from src.load_data import load_pivoted_field

CHECKPOINT_DIR = "../checkpoints/har_pre_model_20250804_140707"

In [7]:
chk = load_all_checkpoints_v2(CHECKPOINT_DIR)
assert isinstance(chk, dict), "Bad checkpoint payload."

ds_test = chk["test"]
X_test = ds_test["X_df"].copy()
y_test = ds_test["y_series"].copy()
model  = chk["model"]

# Rebuild MultiIndex if present
if "idx" in ds_test and ds_test["idx"] is not None:
    idx_df   = ds_test["idx"]
    mi_names = ["timestamp","ticker"] if set(["timestamp","ticker"]).issubset(idx_df.columns) else None
    X_test.index = pd.MultiIndex.from_frame(idx_df, names=mi_names)
    y_test.index = X_test.index

print("Shapes:", X_test.shape, y_test.shape)
print("Model expects:", getattr(model, "feature_names_in_", None))

Shapes: (11020, 4) (11020,)
Model expects: ['signal' 'signal_rank' 'implied' 'implied_rank']


Cell 3 — Feature contract (fail fast, auto-reorder if needed)

In [9]:
feat_in = getattr(model, "feature_names_in_", None)
if feat_in is not None:
    missing = [c for c in feat_in if c not in X_test.columns]
    extra   = [c for c in X_test.columns if c not in feat_in]
    assert not missing, f"Missing features {missing}"
    if extra:
        # Drop unknowns to match training contract
        X_test = X_test.drop(columns=extra)
    # reorder columns
    X_test = X_test.loc[:, list(feat_in)]

# dtype sanity
bad = [c for c in X_test.columns if X_test[c].dtype == "O"]
assert not bad, f"Object dtypes found: {bad}"
assert np.isfinite(X_test.to_numpy()).all(), "NaNs or infs present."

Cell 4 — Quick parity on test (1 metric, 1 print)

In [10]:
def get_scores(m, X):
    # Works for sklearn/xgb/lightgbm; extend if you need torch
    if hasattr(m, "predict_proba"):
        s = m.predict_proba(X)[:, 1]
    elif hasattr(m, "decision_function"):
        s = m.decision_function(X)
    else:
        s = m.predict(X)  # fallback
    return np.asarray(s, dtype=float)

y_score = get_scores(model, X_test)

# Use AUC (don’t threshold y_test if it’s already {0,1})
y_true = np.asarray(y_test, dtype=float)
auc_now = roc_auc_score(y_true, y_score)

auc_then = None
if "metrics" in chk and "auc" in chk["metrics"]:
    auc_then = float(chk["metrics"]["auc"])

print(f"AUC now: {auc_now:.6f}", f"(stored: {auc_then:.6f})" if auc_then is not None else "")

AUC now: 0.747499 


Cell 5 — Build signals → trades (vectorized, with MultiIndex)

In [11]:
# Series with MultiIndex (timestamp,ticker)
scores_s = pd.Series(y_score, index=X_test.index, name="score")
scores_w = scores_s.unstack("ticker").sort_index()  # wide by ticker

# Per-timestamp cross-sectional ranks (top/bottom deciles)
rank_pct = scores_w.rank(axis=1, pct=True, method="average")

buy_mask  = rank_pct >= 0.90
sell_mask = rank_pct <= 0.10

# Vectorized extraction to a tidy trades DataFrame
long = pd.DataFrame({
    "buy":  buy_mask.stack(),
    "sell": sell_mask.stack()
}).reset_index().rename(columns={"level_0":"timestamp","level_1":"ticker"})

long = long[(long["buy"]) | (long["sell"])]
long["action"] = np.where(long["buy"], "buy", "sell")
trades_df = long[["timestamp","ticker","action"]].sort_values(["timestamp","ticker"]).reset_index(drop=True)

print(trades_df.head())
print("n_trades:", len(trades_df))

   timestamp ticker action
0 2023-11-20    AAL   sell
1 2023-11-20    AAP   sell
2 2023-11-20    ABT    buy
3 2023-11-20    ADM    buy
4 2023-11-20    ADP    buy
n_trades: 2210


In [ ]:
Cell 6 — Load new data (IV & close) and align

In [14]:
# Your existing loaders
with open("../configs/config.json") as f:  cfg = json.load(f)
with open("../configs/config.yaml") as f:  cfg_exp = yaml.safe_load(f)

vol = prepare_volatility_data(cfg)          # returns dict with ["implied"] wide matrix
close = load_pivoted_field("close")         # wide close matrix
close.index.rename("date", inplace=True)

# Utility from your snippet
def get_daily_snapshot(date: pd.Timestamp, close_m: pd.DataFrame, iv_m: pd.DataFrame) -> pd.DataFrame:
    close_row = close_m.loc[date]
    iv_row    = iv_m.loc[date]
    tickers   = close_row.index.intersection(iv_row.index)
    df = pd.DataFrame({
        "ticker": tickers,
        "close":  close_row[tickers].values,
        "implied_vol": iv_row[tickers].values
    }).dropna()
    return df

# example day
day = pd.Timestamp("2022-07-28")
spot_iv_df = get_daily_snapshot(day, close, vol["implied"])
spot_iv_df.head()

/Users/stirlitz/ncGitHub/vol_forecast/notebooks/../src/load_data.py:149: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_returns = df_prices.pct_change()


,ticker,close,implied_vol
1,AAL,13.755,0.583418
2,AAP,191.130,0.322366
5,ABC,145.530,0.383099
6,ABT,110.360,0.237346
8,ACN,303.540,0.283147


Cell 7 — Score the new feature matrix and produce trades (optional)

In [15]:
# Build / load X_new somehow (not shown). Then enforce contract:
if feat_in is not None:
    X_new = X_new.reindex(columns=feat_in)
    assert not X_new.columns.isnull().any()
    assert np.isfinite(X_new.to_numpy()).all()

y_score_new = get_scores(model, X_new)
scores_new_s = pd.Series(y_score_new, index=X_new.index, name="score")
scores_new_w = scores_new_s.unstack("ticker").sort_index()

rank_pct_new = scores_new_w.rank(axis=1, pct=True, method="average")
buy_new  = rank_pct_new >= 0.90
sell_new = rank_pct_new <= 0.10

trades_new = (
    pd.DataFrame({"buy": buy_new.stack(), "sell": sell_new.stack()})
      .query("buy or sell")
      .assign(action=lambda d: np.where(d["buy"], "buy","sell"))
      .reset_index().rename(columns={"level_0":"timestamp","level_1":"ticker"})
      [["timestamp","ticker","action"]]
      .sort_values(["timestamp","ticker"])
)
trades_new.head()

NameError: name 'X_new' is not defined

In [23]:
trades=[(row["timestamp"],row["ticker"],row["action"]) for row in trades_df.itertuples()]

TypeError: tuple indices must be integers or slices, not str

In [20]:
ledger = TradeLedger()
generate_straddle_trades(trades_df, close, vol["implied"], ledger)   # or trades_new if using new data

KeyError: 'Missing spot for [(datetime.date(2023, 11, 20), nan), (datetime.date(2023, 11, 20), nan), (datetime.date(2023, 11, 20), nan), (datetime.date(2023, 11, 20), nan), (datetime.date(2023, 11, 20), nan), (datetime.date(2023, 11, 20), nan), (datetime.date(2023, 11, 20), nan), (datetime.date(2023, 11, 20), nan), (datetime.date(2023, 11, 20), nan), (datetime.date(2023, 11, 20), nan), (datetime.date(2023, 11, 20), nan), (datetime.date(2023, 11, 20), nan), (datetime.date(2023, 11, 20), nan), (datetime.date(2023, 11, 20), nan), (datetime.date(2023, 11, 20), nan), (datetime.date(2023, 11, 20), nan), (datetime.date(2023, 11, 20), nan), (datetime.date(2023, 11, 20), nan), (datetime.date(2023, 11, 20), nan), (datetime.date(2023, 11, 20), nan), (datetime.date(2023, 11, 20), nan), (datetime.date(2023, 11, 20), nan), (datetime.date(2023, 11, 20), nan), (datetime.date(2023, 11, 20), nan), (datetime.date(2023, 11, 20), nan), (datetime.date(2023, 11, 20), nan), (datetime.date(2023, 11, 20), nan),

In [22]:
trades_df["timestamp"]=pd.to_datetime(trades_df["timestamp"])
trades_df.iloc[0]


timestamp    2023-11-20 00:00:00
ticker                       AAL
action                      sell
Name: 0, dtype: object